# Workspace for Project Euler problems

## Problem 6

The sum of the squares of the first ten natural numbers is,

$$1^2 + 2^2 + ... + 10^2 = 385$$

The square of the sum of the first ten natural numbers is,

$$(1 + 2 + ... + 10)^2 = 55^2 = 3025$$

Hence the difference between the sum of the squares of the first ten natural numbers and the square of the sum is $3025 − 385 = 2640$.

Find the difference between the sum of the squares of the first one hundred natural numbers and the square of the sum.

This one seems pretty easy I guess

In [2]:
def sumOfSquares(upto):
    "Sum the squares from 1 up to the parameter value"
    sum=0
    for i in range(1, upto+1):
        sum+=(i**2)
    return sum

sumOfSquares(10)

385

In [5]:
def squareOfSums(upto):
    "Square the sum from 1 up to the parameter"
    # The sum of 1 through n is n*(n+1)/2
    return (upto*(upto+1)/2)**2

squareOfSums(10)

3025.0

In [10]:
num=100
sumosq=sumOfSquares(num)
sqofsum=squareOfSums(num)
print ("Sum of squares: {}".format(sumosq))
print ("Square of sums: {}".format(sqofsum))
# apparently it just wants the difference, not the subtraction
print ("Difference: {}".format(abs(sumosq-sqofsum)))

Sum of squares: 338350
Square of sums: 25502500.0
Differnce: 25164150.0
